In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

2023-01-07 07:09:52.784811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [3]:
!jupyter nbconvert --to python ../collect/get_newest_price.ipynb
!python ../collect/get_newest_price.py

[NbConvertApp] Converting notebook ../collect/get_newest_price.ipynb to python
[NbConvertApp] Writing 1547 bytes to ../collect/get_newest_price.py


# Origin data

In [32]:
pd_df = pd.read_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv")
pd_df = pd_df.iloc[-1800:]
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
170513,1671430500000,16722.58,16726.77,16701.00,16712.88,1535.23262,1671431399999,2.565679e+07,35503,651.59573,1.089014e+07
170514,1671431400000,16712.88,16727.71,16697.01,16710.96,1415.62533,1671432299999,2.365662e+07,36183,663.25821,1.108413e+07
170515,1671432300000,16710.45,16750.00,16708.07,16739.36,2191.66513,1671433199999,3.666651e+07,45337,1221.01613,2.042736e+07
170516,1671433200000,16738.84,16748.28,16730.55,16735.15,1523.68618,1671434099999,2.550536e+07,39185,672.25906,1.125346e+07
170517,1671434100000,16735.15,16750.00,16731.41,16744.45,1099.65172,1671434999999,1.840564e+07,31358,526.29445,8.809256e+06
...,...,...,...,...,...,...,...,...,...,...,...
172308,1673046000000,16958.35,16980.00,16944.02,16976.06,2071.22484,1673046899999,3.513468e+07,57575,1062.56575,1.802509e+07
172309,1673046900000,16976.67,16979.99,16964.11,16967.81,1119.96745,1673047799999,1.900905e+07,43663,529.79539,8.992253e+06
172310,1673047800000,16968.04,16969.99,16942.42,16943.20,1307.38421,1673048699999,2.217178e+07,44354,595.92945,1.010676e+07
172311,1673048700000,16943.20,16966.82,16942.10,16950.65,1307.50729,1673049599999,2.217043e+07,40813,664.33823,1.126476e+07


### Format open price and time

In [33]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit='ms') + pd.Timedelta('07:00:00')
pd_df

,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Kline open time,,,,,,,,,,
2022-12-19 13:15:00,16722.58,16726.77,16701.00,16712.88,1535.23262,1671431399999,2.565679e+07,35503,651.59573,1.089014e+07
2022-12-19 13:30:00,16712.88,16727.71,16697.01,16710.96,1415.62533,1671432299999,2.365662e+07,36183,663.25821,1.108413e+07
2022-12-19 13:45:00,16710.45,16750.00,16708.07,16739.36,2191.66513,1671433199999,3.666651e+07,45337,1221.01613,2.042736e+07
2022-12-19 14:00:00,16738.84,16748.28,16730.55,16735.15,1523.68618,1671434099999,2.550536e+07,39185,672.25906,1.125346e+07
2022-12-19 14:15:00,16735.15,16750.00,16731.41,16744.45,1099.65172,1671434999999,1.840564e+07,31358,526.29445,8.809256e+06
...,...,...,...,...,...,...,...,...,...,...
2023-01-07 06:00:00,16958.35,16980.00,16944.02,16976.06,2071.22484,1673046899999,3.513468e+07,57575,1062.56575,1.802509e+07
2023-01-07 06:15:00,16976.67,16979.99,16964.11,16967.81,1119.96745,1673047799999,1.900905e+07,43663,529.79539,8.992253e+06
2023-01-07 06:30:00,16968.04,16969.99,16942.42,16943.20,1307.38421,1673048699999,2.217178e+07,44354,595.92945,1.010676e+07


In [34]:
fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
                open=pd_df['Open price'],
                high=pd_df['High price'],
                low=pd_df['Low price'],
                close=pd_df['Close price'])])
# set new height and width
fig.update_layout(
    height=800,
    width=1000,
    title_text="BTC/USDT price",
    yaxis_title="Price (BTC/USDT)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=True
)

fig.show()

# Prepare dataset

### Price column

In [59]:
dataset = pd_df.filter(["Open price"]).values
# dataset = dataset[int(dataset.shape[0] * 0.8):]
dataset.shape

(1800, 1)

### Splitting ratio

In [60]:
TRAIN_DATA_LENGTH = int(len(dataset) * 0.8)

### Scale data to [0, 1]

In [61]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.56792802],
       [0.5517368 ],
       [0.54768065],
       ...,
       [0.97764943],
       [0.93618655],
       [0.94805455]])

### Split training set

In [62]:
train_data = scaled_data[:TRAIN_DATA_LENGTH]

X_train = []
Y_train = []
for i in range(60,len(train_data)):
    X_train.append(train_data[i-60:i,:])
    Y_train.append(train_data[i,:])

X_train, Y_train = np.array(X_train), np.array(Y_train)
print(X_train.shape, Y_train.shape)

(1380, 60, 1) (1380, 1)


### Split test set

In [63]:
test_data = scaled_data[TRAIN_DATA_LENGTH-60:]

X_test = []
Y_test = []
for i in range(60,len(test_data)):
    X_test.append(test_data[i-60:i,:])
    Y_test.append(test_data[i,:])

X_test, Y_test = np.array(X_test), np.array(Y_test)
print(X_test.shape, Y_test.shape)

(360, 60, 1) (360, 1)


# Training

## Long-short term memory model

In [64]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(100))
model.add(Dense(X_train.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=1, epochs=1)

1380/1380 [==============================] - 30s 20ms/step - loss: 0.0035


### Predict and transform [0, 1] to the original scale

In [69]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions[:5]

12/12 [==============================] - 0s 30ms/step


array([[16723.062],
       [16734.455],
       [16750.611],
       [16744.98 ],
       [16736.857]], dtype=float32)

### Visualize the prediction

In [70]:
data = pd_df.filter(["Open price"])
data.shape

(1800, 1)

In [71]:
train = data[:TRAIN_DATA_LENGTH]
valid = data[TRAIN_DATA_LENGTH:].reset_index()
valid["Predict"] = predictions[:, 0]
valid = valid.set_index("Kline open time")
valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

lstm_df = pd.concat([train, valid], axis=0)

In [72]:
fig = px.line(lstm_df[["Open price","Predict"]], title="BTC/USDT price" , width=1000, height=800)
fig.show()

## Autoregressive integrated moving average model

In [73]:
from statsmodels.tsa.arima.model import ARIMA

### Scale data to [0, 1] and split train-test set

In [74]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]
test.shape

(360, 1)

### Train model

In [ ]:
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

### Visualize the prediction

In [81]:
train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)
arima_df = pd.concat([train_df, test_df], axis=0)


fig = px.line(arima_df, title="BTC/USDT price" , width=1000, height=800)
fig.show()               

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_3463/3839462130.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Model selection with cross validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from estimator import ArimaEstimator, LSTMEstimator, SarimaxEstimator, AutoRegEstimator, ExponentialSmoothingEstimator, HoltEstimator

# cross vadiation strategy
tscv = TimeSeriesSplit(n_splits=5)

# mean squared error metric
metric = make_scorer(mean_squared_error)

# models
estimators = [
    ArimaEstimator(), 
    SarimaxEstimator(), 
    AutoRegEstimator(),
    ExponentialSmoothingEstimator(), 
    HoltEstimator(),
    LSTMEstimator(), 
]

scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_train, Y_train, scoring=metric, cv=tscv, n_jobs=-1)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'fold_{x+1}')
scores_df.loc['mean'] = scores_df.mean()
scores_df.to_csv('../../datastore/scores.csv')

In [111]:
scores_df

,ArimaEstimator,SarimaxEstimator,AutoRegEstimator,ExponentialSmoothingEstimator,HoltEstimator,LSTMEstimator
fold_1,0.000027,0.000032,0.000022,0.000020,0.000022,0.000558
fold_2,0.000027,0.000050,0.000025,0.000024,0.000026,0.000038
fold_3,0.000036,0.000052,0.000034,0.000031,0.000033,0.000064
fold_4,0.000840,0.001099,0.000818,0.000777,0.000807,0.004770
fold_5,0.000769,0.001182,0.000754,0.000696,0.000722,0.003198
mean,0.000340,0.000483,0.000331,0.000310,0.000322,0.001726


In [112]:
# find column with lowest mean score
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'ExponentialSmoothingEstimator'

In [ ]:
%%capture
from estimator import ExponentialSmoothingEstimator as Estimator
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

tscv = TimeSeriesSplit()

metric = make_scorer(mean_squared_error, greater_is_better=False)

param_grid = {
    'trend': ["add", "mul", "additive", "multiplicative", None],
    'damped_trend': [True, False],
    'seasonal': ["add", "mul", "additive", "multiplicative", None],
    'seasonal_periods': [7, 30, 365],
}

estimator = Estimator()
estimator.get_params().keys()
grid_search = GridSearchCV(estimator, param_grid, scoring=metric, cv=tscv, n_jobs=-1, verbose=0)
grid_search.fit(X_train, Y_train)

In [114]:
grid_search.best_estimator_

ExponentialSmoothingEstimator(seasonal_periods=7)

# Params selection with grid search

In [115]:
from statsmodels.tsa.api import ExponentialSmoothing

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ExponentialSmoothing(history, seasonal_periods=7)
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

In [116]:

train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)


# concate train_df and test_df to get the full data set
full_df = pd.concat([train_df, test_df], axis=0)


fig = px.line(full_df, height=800, width=1000)
fig.show()

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_2422/704318073.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [117]:
model = ExponentialSmoothing(data["Open price"].values, seasonal_periods=7)
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]

In [118]:
pred_price

16832.28719995541

In [119]:
data

,Open price
Kline open time,
2018-02-02,9224.52
2018-02-03,8873.03
2018-02-04,9199.96
2018-02-05,8179.99
2018-02-06,6939.63
...,...
2023-01-02,16617.17
2023-01-03,16672.78
2023-01-04,16675.65
